In [1]:
## imports
import pandas as pd
import numpy as np
import mne
import re
import os

from mne.preprocessing import (
    ICA, 
    create_ecg_epochs, 
    corrmap, 
    create_eog_epochs
)

# mne graphics config
# from PySide6 import QtCore, QtGui, QtWidgets, __version__ 

rawa = None
rawb = None

import datetime
print('IMPORTED MODULES today:', datetime.datetime.now())

# Removed cached function import
import sys
try:
    del sys.modules['rename_events']
except KeyError:
    pass

from rename_events import relabel_events


# setup

# SESSION AND POSNER CODES
session_id='m_15_02'
posner_id ='pos2a'
posner_id_b ='pos2b'

IMPORTED MODULES today: 2023-07-18 17:03:21.419352


In [2]:
%matplotlib qt

# Posner A

In [3]:
# Relabel stim events and load raw data
raw_a = relabel_events(session_id, posner_id)

Data file:  m_15_02/m_15_02_pos2a.vhdr


/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


Used Annotations descriptions: ['New Segment/', 'Stimulus/S 20', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109', 'Stimulus/S117']
Found '2a' in file: m_15_02/15_2a_posner_task_2023-06-09_17h12.13.155/15_2a_posner_task_2023-06-09_17h12.13.155.csv
Found '2b' in file: m_15_02/15_2b_posner_task_2023-06-09_18h10.06.059/15_2b_posner_task_2023-06-09_18h10.06.059.csv
********** 2a
Used Annotations descriptions: ['New Segment/', 'Stimulus/S 20', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109', 'Stimulus/S117']
Cropping annotations 2023-06-09 17:15:06.310215+00:00 - 2023-06-09 17:26:28.230215+00:00
  [0] Keeping  (2023-06-09 17:15:06.310215+00:00 - 2023-06-09 17:15:06.310215+00:00 -> 0.0 - 0.0)
  [1] Keeping  (2023-06-09 17:15:10.385215+00:00 - 2023-06-09 17:15:10.385215+00:00 -> 4.075 - 4.

/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


  [365] Keeping  (2023-06-09 17:25:00.432215+00:00 - 2023-06-09 17:25:00.432215+00:00 -> 594.122 - 594.122)
  [366] Keeping  (2023-06-09 17:25:01.932215+00:00 - 2023-06-09 17:25:01.932215+00:00 -> 595.622 - 595.622)
  [367] Keeping  (2023-06-09 17:25:04.622215+00:00 - 2023-06-09 17:25:04.622215+00:00 -> 598.312 - 598.312)
  [368] Keeping  (2023-06-09 17:25:04.722215+00:00 - 2023-06-09 17:25:04.722215+00:00 -> 598.412 - 598.412)
  [369] Keeping  (2023-06-09 17:25:06.932215+00:00 - 2023-06-09 17:25:06.932215+00:00 -> 600.622 - 600.622)
  [370] Keeping  (2023-06-09 17:25:08.432215+00:00 - 2023-06-09 17:25:08.432215+00:00 -> 602.122 - 602.122)
  [371] Keeping  (2023-06-09 17:25:11.662215+00:00 - 2023-06-09 17:25:11.662215+00:00 -> 605.352 - 605.352)
  [372] Keeping  (2023-06-09 17:25:11.762215+00:00 - 2023-06-09 17:25:11.762215+00:00 -> 605.452 - 605.452)
  [373] Keeping  (2023-06-09 17:25:13.432215+00:00 - 2023-06-09 17:25:13.432215+00:00 -> 607.122 - 607.122)
  [374] Keeping  (2023-06-09

In [5]:
event_map = {'1': 11, '10': 10, '101': 101, '109': 109, '11': 11, '117': 117, '20': 20, '30': 30, '40': 40, '5': 5, '50': 50, '51': 51, '52': 52, '60': 60, '70': 70, '80': 80, '99999': 99999}

# get events and times
events, event_id = mne.events_from_annotations(raw_a, event_map)

e_a = events
# all rows with fixation (code 40)
e_40_a = e_a[e_a[:, 2] == 40]
# all rows with trial end (code 101)
e_101_a = e_a[e_a[:,2]==101]

a_tmin= e_40_a[0][0]/1000
a_tmax= e_101_a[-1][0]/1000

# Crop and make a copy and filter
filt_raw_a = raw_a.crop(tmin=a_tmin, tmax=a_tmax).copy().filter(l_freq=1.0, h_freq=40)

epochs_a = mne.Epochs(filt_raw_a, 
                    e_a, 
                    event_id=None, 
                    tmin=-0.2, 
                    tmax=0.5, 
                    preload=True, 
                    baseline=(None, 0))

Used Annotations descriptions: ['101', '109', '30', '40', '50', '51', '52', '60', '70', '80']


ValueError: tmax (667.511) must be less than or equal to the max time (663.4360 s)

In [50]:
raw_a.plot()

Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [6]:
# Fitting ICA
ica = ICA(n_components=len(filt_raw_a.ch_names)-1, max_iter="auto", random_state=999)
ica.fit(filt_raw_a)
ica

Fitting ICA to data using 63 channels (please be patient, this may take a while)
Selecting by number: 63 components
Fitting ICA took 78.7s.


Method,fastica
Fit,125 iterations on raw data (663437 samples)
ICA components,63
Available PCA components,63
Channel types,eeg
ICA components marked for exclusion,—


In [7]:
ica.plot_components()

[<MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 585x260 with 3 Axes>]

In [ ]:
raw_a.load_data()
ica.plot_sources(raw_a)

Creating RawArray with float64 data, n_channels=63, n_times=663437
    Range : 4075 ... 667511 =      4.075 ...   667.511 secs
Ready.
Using qt as 2D backend.
Using pyopengl with version 3.1.6


    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
331 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
331 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
331 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
331 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
331 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
331 matching events found
No baseline correction applied
0 projection items ac

In [8]:
print(ica.exclude)

[44, 42, 47, 31, 32, 27, 24, 25, 1, 3, 7, 15, 9]


In [9]:
reconstr_raw = raw_a.copy()
# Apply ica to copy
ica.apply(reconstr_raw)
# Plot
raw_a.plot(title="Raw data")
reconstr_raw.plot(title="ICA applied")

Applying ICA to Raw instance
    Transforming to ICA space (63 components)
    Zeroing out 13 ICA components
    Projecting back using 63 PCA components
Using qt as 2D backend.
Using pyopengl with version 3.1.6
Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [10]:
# Save file
file_name = session_id+'_'+posner_id+'.vhdr'
output_dir='cleaned'

print(file_name)
cur_dir = os.getcwd()
session_dir = os.path.join(cur_dir, output_dir, session_id)
print(session_dir)

if not os.path.exists(session_dir):
   os.makedirs(session_dir)

print("\n**** Saving file to: ", session_dir+'/'+file_name) 
mne.export.export_raw(session_dir+'/'+file_name, reconstr_raw, verbose=1)

m_15_02_pos2a.vhdr
/Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_15_02

**** Saving file to:  /Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_15_02/m_15_02_pos2a.vhdr


/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/_export.py:61: RuntimeWarning: Encountered data in 'short' format. Converting to float32.
  warn(
/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/io.py:690: UserWarning: Encountered unsupported non-voltage units: n/a
Note that the BrainVision format specification supports only µV.
  warn(msg)


In [11]:
!tree {session_dir}

/Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_15_02
├── m_15_02_pos2a.eeg
├── m_15_02_pos2a.vhdr
└── m_15_02_pos2a.vmrk

1 directory, 3 files


In [12]:
del raw_a
del reconstr_raw
del ica
del filt_raw_a

# Posner B

In [13]:
# Relabel stim events and load raw data
raw_b = relabel_events(session_id, posner_id_b)

Data file:  m_15_02/m_15_02_pos2b.vhdr


/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 20', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109']
Found '2a' in file: m_15_02/15_2a_posner_task_2023-06-09_17h12.13.155/15_2a_posner_task_2023-06-09_17h12.13.155.csv
Found '2b' in file: m_15_02/15_2b_posner_task_2023-06-09_18h10.06.059/15_2b_posner_task_2023-06-09_18h10.06.059.csv
********** 2b
Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 20', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109']
Cropping annotations 2023-06-09 18:13:09.397011+00:00 - 2023-06-09 18:25:01.777011+00:00
  [0] Keeping  (2023-06-09 18:13:09.397011+00:00 - 2023-06-09 18:13:09.397011+00:00 -> 0.0 - 0.0)
  [1] Keeping  (2023-06-09 18:13:12.215011+00:00 - 2023-06-09 18:13:12.215011+00:00 -> 2.818 - 2.

/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


  [352] Keeping  (2023-06-09 18:23:09.910011+00:00 - 2023-06-09 18:23:09.910011+00:00 -> 600.513 - 600.513)
  [353] Keeping  (2023-06-09 18:23:10.010011+00:00 - 2023-06-09 18:23:10.010011+00:00 -> 600.613 - 600.613)
  [354] Keeping  (2023-06-09 18:23:11.460011+00:00 - 2023-06-09 18:23:11.460011+00:00 -> 602.063 - 602.063)
  [355] Keeping  (2023-06-09 18:23:12.960011+00:00 - 2023-06-09 18:23:12.960011+00:00 -> 603.563 - 603.563)
  [356] Keeping  (2023-06-09 18:23:16.330011+00:00 - 2023-06-09 18:23:16.330011+00:00 -> 606.933 - 606.933)
  [357] Keeping  (2023-06-09 18:23:16.430011+00:00 - 2023-06-09 18:23:16.430011+00:00 -> 607.033 - 607.033)
  [358] Keeping  (2023-06-09 18:23:17.961011+00:00 - 2023-06-09 18:23:17.961011+00:00 -> 608.564 - 608.564)
  [359] Keeping  (2023-06-09 18:23:19.460011+00:00 - 2023-06-09 18:23:19.460011+00:00 -> 610.063 - 610.063)
  [360] Keeping  (2023-06-09 18:23:23.360011+00:00 - 2023-06-09 18:23:23.360011+00:00 -> 613.963 - 613.963)
  [361] Keeping  (2023-06-09

In [14]:
event_map = {'1': 11, '10': 10, '101': 101, '109': 109, '11': 11, '117': 117, '20': 20, '30': 30, '40': 40, '5': 5, '50': 50, '51': 51, '52': 52, '60': 60, '70': 70, '80': 80, '99999': 99999}

# get events and times
events_b, event_id_b = mne.events_from_annotations(raw_b, event_map)

e_b = events_b
# all rows with fixation (code 40)
e_40_b = e_b[e_b[:, 2] == 40]
# all rows with trial end (code 101)
e_101_b = e_b[e_b[:,2]==101]

b_tmin= e_40_b[0][0]/1000
b_tmax= e_101_b[-1][0]/1000

# Crop and make a copy and filter
filt_raw_b = raw_b.crop(tmin=b_tmin, tmax=b_tmax).copy().filter(l_freq=1.0, h_freq=40)

epochs_b = mne.Epochs(filt_raw_b, 
                    e_b, 
                    event_id=None, 
                    tmin=-0.2, 
                    tmax=0.5, 
                    preload=True, 
                    baseline=(None, 0))

Used Annotations descriptions: ['10', '101', '109', '20', '30', '40', '50', '51', '52', '60', '70', '80', '99999']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Not setting metadata
413 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 413 events and 701 original time points ...


[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.7s finished


5 bad epochs dropped


In [5]:
raw_b.plot()

Using qt as 2D backend.
Using pyopengl with version 3.1.6


Channels marked as bad:
none


In [15]:
# Fitting ICA
ica_b = ICA(n_components=len(filt_raw_b.ch_names)-1, max_iter="auto", random_state=999)
ica_b.fit(filt_raw_b, verbose='DEBUG')
ica_b

Fitting ICA to data using 63 channels (please be patient, this may take a while)
Selecting by number: 63 components
Cropping annotations 2023-06-09 18:13:16.757011+00:00 - 2023-06-09 18:24:44.348011+00:00
  [0] Keeping  (2023-06-09 18:13:16.757011+00:00 - 2023-06-09 18:13:16.757011+00:00 -> 7.36 - 7.36)
  [1] Keeping  (2023-06-09 18:13:18.250011+00:00 - 2023-06-09 18:13:18.250011+00:00 -> 8.853 - 8.853)
  [2] Keeping  (2023-06-09 18:13:21.367011+00:00 - 2023-06-09 18:13:21.367011+00:00 -> 11.97 - 11.97)
  [3] Keeping  (2023-06-09 18:13:21.467011+00:00 - 2023-06-09 18:13:21.467011+00:00 -> 12.07 - 12.07)
  [4] Keeping  (2023-06-09 18:13:23.257011+00:00 - 2023-06-09 18:13:23.257011+00:00 -> 13.86 - 13.86)
  [5] Keeping  (2023-06-09 18:13:24.757011+00:00 - 2023-06-09 18:13:24.757011+00:00 -> 15.36 - 15.36)
  [6] Keeping  (2023-06-09 18:13:27.627011+00:00 - 2023-06-09 18:13:27.627011+00:00 -> 18.23 - 18.23)
  [7] Keeping  (2023-06-09 18:13:27.727011+00:00 - 2023-06-09 18:13:27.727011+00:00

Method,fastica
Fit,58 iterations on raw data (687591 samples)
ICA components,63
Available PCA components,63
Channel types,eeg
ICA components marked for exclusion,—


In [16]:
ica_b.plot_components()

[<MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 585x260 with 3 Axes>]

In [ ]:
raw_b.load_data()
ica_b.plot_sources(raw_b)

Creating RawArray with float64 data, n_channels=63, n_times=687591
    Range : 7360 ... 694950 =      7.360 ...   694.950 secs
Ready.
Using pyopengl with version 3.1.6


    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
343 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
343 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
343 matching events found
No baseline correction applied
0 projection items activated


In [17]:
ica_b.exclude

[41, 45, 50, 58, 54, 22, 26, 1, 6, 5, 4, 15, 11]

In [18]:
reconstr_raw_b = raw_b.copy()
# Apply ica to copy
ica_b.apply(reconstr_raw_b)
# Plot and compare
raw_b.plot(title="Raw data (posner b)")
reconstr_raw_b.plot(title="ICA applied (posner b)")

Applying ICA to Raw instance
    Transforming to ICA space (63 components)
    Zeroing out 13 ICA components
    Projecting back using 63 PCA components
Using pyopengl with version 3.1.6
Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [19]:
# Save file
file_name_b = session_id+'_'+posner_id_b+'.vhdr'
output_dir_b='cleaned'
cur_dir = os.getcwd()
session_dir = os.path.join(cur_dir, output_dir_b, session_id)
if not os.path.exists(session_dir):
   os.makedirs(session_dir)

print("\n**** Saving file to: ", session_dir+'/'+file_name_b) 
mne.export.export_raw(session_dir+'/'+file_name_b, reconstr_raw_b, verbose=1, overwrite=True)


**** Saving file to:  /Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_15_02/m_15_02_pos2b.vhdr


/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/_export.py:61: RuntimeWarning: Encountered data in 'short' format. Converting to float32.
  warn(
/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/io.py:690: UserWarning: Encountered unsupported non-voltage units: n/a
Note that the BrainVision format specification supports only µV.
  warn(msg)


In [20]:
!tree {session_dir}

/Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_15_02
├── m_15_02_pos2a.eeg
├── m_15_02_pos2a.vhdr
├── m_15_02_pos2a.vmrk
├── m_15_02_pos2b.eeg
├── m_15_02_pos2b.vhdr
└── m_15_02_pos2b.vmrk

1 directory, 6 files


# ...

In [21]:
del raw_b
del reconstr_raw_b
del ica_b
del filt_raw_b